This is the first script in the Inception Pipeline. It identifes best features subsets for each of the 6 categories for which we have human judgments. These subsets are later used to produce modified models from which we compute FID and IS scores. 

# 1.Preparation

Useful libraries 

In [ ]:
import os # for file handling
import torch # for deep learning
from PIL import Image # for image handling
from torchvision import models, transforms # for pre-trained models
import torch.nn as nn # for neural network layers
import numpy as np # for numerical operations
from scipy.stats import spearmanr # for correlation calculation

Understanding the dataset structure

In [ ]:
import os

images_path = f'{os.getcwd()}/Peterson_data/images_by_categories'
# print the images in the folder
categories = [f for f in os.listdir(images_path) if os.path.isdir(os.path.join(images_path, f))]
print(categories)

## checks for CUDA/GPU avail

In [ ]:
import torch

# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA Available: {cuda_available}")
# Check the number of GPUs
num_gpus = torch.cuda.device_count()
print(f"Number of GPUs: {num_gpus}")

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

# Print the CUDA version
print(f"CUDA Version: {torch.version.cuda}")


In [ ]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    # Create a tensor and move it to GPU
    tensor = torch.tensor([1.0, 2.0, 3.0])
    tensor_gpu = tensor.to('cuda')
    print(f"Tensor on GPU: {tensor_gpu}")
else:
    print("CUDA is not available. Tensor cannot be moved to GPU.")
import torch

# Check if the installed PyTorch version is compatible with the installed CUDA version
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Version: {torch.version.cuda}")
print(f"cuDNN Version: {torch.backends.cudnn.version()}")


## functions

In [ ]:
def upper_triangle_values(matrix):
    """
    Extracts the upper triangular values from a square matrix, excluding the diagonal.

    Args:
    matrix (numpy.ndarray): A 2D square numpy array.

    Returns:
    numpy.ndarray: A 1D numpy array containing the upper triangular values of the input matrix.
    """
    # Extract the upper triangle indices, excluding the diagonal
    upper_triangle_indices = np.triu_indices_from(matrix, k=1)
    return matrix[upper_triangle_indices]


def compute_spearman_uppert(matrix1, matrix2):
    """
    Computes the Spearman correlation between the upper triangles of two matrices.

    Args:
    matrix1 (numpy.ndarray): The first 2D square numpy array.
    matrix2 (numpy.ndarray): The second 2D square numpy array.

    Returns:
    float: The Spearman correlation coefficient between the upper triangles of the input matrices.
    """
    upper_triangle_values1 = upper_triangle_values(matrix1)
    upper_triangle_values2 = upper_triangle_values(matrix2)
    correlation, _ = spearmanr(upper_triangle_values1, upper_triangle_values2)
    return correlation


def compute_cosine_similarity_matrix(embeddings):
    """
    Computes the cosine similarity matrix for a given array of embeddings.

    Args:
    embeddings (numpy.ndarray): A 2D numpy array of shape (n_samples, n_features) representing the embeddings.

    Returns:
    numpy.ndarray: A 2D numpy array of shape (n_samples, n_samples) representing the cosine similarity matrix.
    """
    norms = np.linalg.norm(embeddings, axis=1)
    dot_product = np.dot(embeddings, embeddings.T)
    similarity_matrix = dot_product / (norms[:, None] * norms[None, :])
    return similarity_matrix



# 2.load model and create embmeddings from full model

1. SET the category here
2. using pool_3 layer for embeddings
2.   do not touch the normalization as this is how images are normalized for training inception and has to be kept the same for evaluation of new images
3. the model mimicks the inception pipeline but ends it at the pool_3 layer to extract embeddings




In [ ]:
# Define the category variable: animals, automobiles, furniture, fruits, vegetables, various

category = 'vegetables'

# Use the category variable in the path and dictionary key



# Define a transformation to preprocess the images
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Resize images to 299x299 for Inception-v3
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

# In this function, need to sort the filenames to match the human similairty judgments order
# get a printout of filenames to make sure these are sorted well
def load_images_from_directory(image_dir, transform):
    images = []
    filenames = sorted([f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))])
    print(filenames)
    for filename in filenames:
        img_path = os.path.join(image_dir, filename)
        image = Image.open(img_path).convert('RGB')
        image = transform(image)
        images.append(image)
    return torch.stack(images)


class InceptionV3Pool3(nn.Module):
    def __init__(self):
        super(InceptionV3Pool3, self).__init__()
        inception = models.inception_v3(pretrained=True)
        self.features = nn.Sequential(
            inception.Conv2d_1a_3x3,
            inception.Conv2d_2a_3x3,
            inception.Conv2d_2b_3x3,
            nn.MaxPool2d(kernel_size=3, stride=2),
            inception.Conv2d_3b_1x1,
            inception.Conv2d_4a_3x3,
            nn.MaxPool2d(kernel_size=3, stride=2),
            inception.Mixed_5b,
            inception.Mixed_5c,
            inception.Mixed_5d,
            inception.Mixed_6a,
            inception.Mixed_6b,
            inception.Mixed_6c,
            inception.Mixed_6d,
            inception.Mixed_6e,
            inception.Mixed_7a,
            inception.Mixed_7b,
            inception.Mixed_7c,
        )
        self.pool3 = nn.AdaptiveAvgPool2d(output_size=(1, 1))

    def forward(self, x):
        x = self.features(x)
        x = self.pool3(x)
        x = torch.flatten(x, 1)
        return x


embedding_model = InceptionV3Pool3()
embedding_model.eval()  # Set the model to evaluation mode


# Load images from the directory
image_dir = image_dir = f'/home/hasson/data/peterson/{category}/images'
images = load_images_from_directory(image_dir, transform)

# Move images to the same device as the model (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

embedding_model.to(device)
images = images.to(device)

# Disable gradient calculations
with torch.no_grad():
    embeddings = embedding_model(images)

# Move embeddings back to CPU if necessary
embeddings = embeddings.cpu()

## Load human sim and compute Baseline correlation

Computation of baseline correlation is for sanity.
It is also computed in the application of the pruning algorithm itself.

In [ ]:
# load human sim judg
# the npz file is an array that holds similarity judgmetns for all categories
hsim = np.load('/home/hasson/data/peterson/hsim_peterson.npz')

# e.g. get animals
hsim_cat = hsim[category]

embeddings_np = embeddings.cpu().numpy()

# Compute cosine similarity matrix
similarity_matrix = compute_cosine_similarity_matrix(embeddings_np)


# Compute the Spearman correlation
spearman_corr = compute_spearman_uppert(hsim_cat, similarity_matrix)

# Print the Spearman correlation coefficient
print(f"Spearman Correlation: {spearman_corr}")



# 3.Supervised pruning

The best_subset is a list of features that will then be used to filter embeddings from inceptionV3 from the human similarity matrices for each category.bold text

In [ ]:
import numpy as np
from scipy.stats import spearmanr
from scipy.spatial.distance import cosine


# Step 1: Compute baseline correlation
def compute_baseline_corr(SM_HM, SM_DNN):
    # Flatten upper triangle of matrices to 1darray
    mask = np.triu(np.ones_like(SM_HM), k=1).astype(bool)
    hm_flat = SM_HM[mask]
    dnn_flat = SM_DNN[mask]
    return spearmanr(hm_flat, dnn_flat)[0]

# Step 2: Rank features
def rank_features(SM_HM, SM_DNN, full_embeddings):
    N = full_embeddings.shape[1]
    baseline_corr = compute_baseline_corr(SM_HM, SM_DNN)

    feature_importance = []
    for i in range(N):
        # print(f"computing feature {i}")
        # Remove feature i
        reduced_embeddings = np.delete(full_embeddings, i, axis=1)
        # Compute cosine similarity matrix
        cosine_sim_matrix = compute_cosine_similarity_matrix(reduced_embeddings)
        # Compute correlation with cosine similarity matrix
        corr_reduced = compute_corr_upper_triangle(SM_HM, cosine_sim_matrix)
        # Calculate difference D; difference in Spearman R
        D = baseline_corr - corr_reduced
        feature_importance.append((i, D))
    # Sort features by importance (D value)
    feature_importance.sort(key=lambda x: x[1], reverse=True)
    return feature_importance

# Function to compute correlation of upper triangle
def compute_corr_upper_triangle(mat1, mat2):
    mask = np.triu(np.ones_like(mat1), k=1).astype(bool)
    mat1_flat = mat1[mask]
    mat2_flat = mat2[mask]
    return spearmanr(mat1_flat, mat2_flat)[0]

# Step 3: Construct pruned embeddings
def construct_pruned_embeddings(SM_HM, full_embeddings, feature_importance):
    best_corr = -np.inf
    best_subset = []

    # Iterate over ranked features
    for i in range(len(feature_importance)):
        # Include features in descending order of importance
        features_to_include = [feature_importance[j][0] for j in range(i + 1)]
        pruned_embeddings = full_embeddings[:, features_to_include]
        cosine_sim_matrix = compute_cosine_similarity_matrix(pruned_embeddings)
        # Compute correlation after including selected features
        corr = compute_corr_upper_triangle(SM_HM, cosine_sim_matrix)
        # Store the maximum correlation and corresponding subset of features
        if corr > best_corr:
            best_corr = corr
            best_subset = features_to_include

    return best_subset

### actual function calls

In [ ]:
SM_HM = hsim_cat
SM_DNN = similarity_matrix
full_embeddings = embeddings_np

# Step 1: Compute baseline correlation
baseline_corr = compute_baseline_corr(SM_HM, SM_DNN)
print(f"baseline correlation is {baseline_corr}")

# Step 2: Rank features
feature_importance = rank_features(SM_HM,SM_DNN,full_embeddings)

# Step 3: Construct pruned embeddings
# best_subset is constructed for each category and will be applied to full embeddings
# can be saved to google drive to be used in future scripts
best_subset = construct_pruned_embeddings(SM_HM, full_embeddings, feature_importance)

print("Best subset of features:", best_subset)

# Step 4: Evaluate pruned embeddings
pruned_embeddings_best = full_embeddings[:, best_subset]
best_prune_sim = compute_cosine_similarity_matrix(pruned_embeddings_best)
best_cor = compute_corr_upper_triangle(SM_HM, best_prune_sim)

baseline_corr = compute_baseline_corr(SM_HM, SM_DNN)
print(f"baseline correlation is {baseline_corr}")
print("Best correlation :", best_cor)

## save best subset list to file

Make sure to change the path here.

In [ ]:
import pandas as pd

# Convert the list to a DataFrame
df = pd.DataFrame(best_subset)

# Define the file name using the category variable
file_name = f"{category}_bestsubset.csv"

# Save the DataFrame as a CSV file to the specified path
df.to_csv(f'/home/hasson/progproj/2024Giulia/pruned_subsets/{file_name}', index=False, header=False)
